In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
from astropy import units
import os
import astropy.units as u
import matplotlib.pyplot as plt

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Radial distribution of satellites of M31 and MW

Count the accumulation of satellites with radius from the centres of potential of the MW and M31 galaxies **separately**.

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

## Get data for plotting

Choose how to distinguish between satellites and isolated:

In [ ]:
distinction = "by_gn"

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
def arrange_radius_array(r):
    r = r * units.cm.to(units.kpc)
    r = np.concatenate((np.sort(r), np.array([1000]))) 
    return r

def construct_count_array(size):
    return np.concatenate((np.arange(1, size+1), 
                           np.array([size])))

In [ ]:
for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential")
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "by_r":
        mask_sat, mask_isol = dataset_compute.mask_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_gn":
        mask_sat, mask_isol = dataset_compute.mask_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    
    mask_m31_lum = np.logical_and.reduce([mask_sat[0], mask_lum, mask_nonzero_vmax])
    mask_m31_dark = np.logical_and.reduce([mask_sat[0], mask_dark, mask_nonzero_vmax])
    mask_mw_lum = np.logical_and.reduce([mask_sat[1], mask_lum, mask_nonzero_vmax])
    mask_mw_dark = np.logical_and.reduce([mask_sat[1], mask_dark, mask_nonzero_vmax])
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_mw_lum = np.logical_and(mask_mw_lum, np.logical_not(mask_m31_lum))
    mask_mw_dark = np.logical_and(mask_mw_dark, np.logical_not(mask_m31_dark))
   
    # Add M31 data to the dictionary:
    m31_cop = cops[snap.index_of_halo(sim_data["M31_identifier"][0],
                                      sim_data["M31_identifier"][1])]
    dist_to_m31 = dataset_compute.distance_to_point(snap, m31_cop)
    sim_data["M31_satellites"] = \
    {"radius": {"luminous": arrange_radius_array(
                            dist_to_m31[mask_m31_lum]),
                "dark": arrange_radius_array(
                        dist_to_m31[mask_m31_dark])
               },
     "sat_count": {"luminous": construct_count_array(np.sum(mask_m31_lum)),
                   "dark": construct_count_array(np.sum(mask_m31_dark))
                  }
    }
    
    # Add MW data to the dictionary:
    mw_cop = cops[snap.index_of_halo(sim_data["MW_identifier"][0],
                                     sim_data["MW_identifier"][1])]
    dist_to_mw = dataset_compute.distance_to_point(snap, mw_cop)
    sim_data["MW_satellites"] = \
    {"radius": {"luminous": arrange_radius_array(
                            dist_to_mw[mask_mw_lum]),
                "dark": arrange_radius_array(
                        dist_to_mw[mask_mw_dark])
               },
     "sat_count": {"luminous": construct_count_array(np.sum(mask_mw_lum)),
                   "dark": construct_count_array(np.sum(mask_mw_dark))
                  }
    }

## Plot

In [ ]:
# Set some parameters:
x_down = 0; x_up = 1000
y_down = 1; y_up = 500

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'sat_r_dist_M31andMW_{}'.format(distinction)
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis:
for ax in axes:
    ax.set_yscale('log')
    ax.set_xlim(x_down, x_up)
    ax.set_ylim(y_down, y_up)
    ax.set_xlabel('$r[\mathrm{kpc}]$', fontsize=16)
    ax.set_ylabel('$N(<r)$', fontsize=16)
    
axes[0].set_title('Andromeda')
axes[1].set_title('Milky Way')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['M31_satellites']['radius']['luminous']
    y = entry['M31_satellites']['sat_count']['luminous']
    axes[0].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['M31_satellites']['radius']['dark']
    y = entry['M31_satellites']['sat_count']['dark']
    axes[0].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))
    
    x = entry['MW_satellites']['radius']['luminous']
    y = entry['MW_satellites']['sat_count']['luminous']
    axes[1].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['MW_satellites']['radius']['dark']
    y = entry['MW_satellites']['sat_count']['dark']
    axes[1].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))

axes[0].legend(loc='upper right')
plt.tight_layout()

plt.savefig(filename, dpi=200)